In [1]:
from vnpy.trader.setting import SETTINGS
from vnpy.trader.utility import load_json, save_json

In [2]:
# 更新配置文件数据库名称
database_name = "恒生指数1分钟数据.db"

setting = load_json("vt_setting.json")
setting.update({"database.database": database_name})
save_json("vt_setting.json", setting)

In [3]:
from vnpy.trader.engine import MainEngine, EventEngine
from vnpy.app.data_manager.engine import ManagerEngine

manager = ManagerEngine(MainEngine, EventEngine)

在使用天勤量化之前，默认您已经知晓并同意以下免责条款，如果不同意请立即停止使用：https://www.shinnytech.com/blog/disclaimer/


In [4]:
manager.import_data_from_csv(
        file_path = "恒指一分钟数据.csv",
        symbol    = "HZ",
        exchange  = "HKEX",
        interval  = "1m",
        datetime_head = "datetime",
        open_head   = "open_price",
        high_head   = "high_price",
        low_head    = "low_price",
        close_head  = "close_price",
        volume_head = "volume",
        open_interest_head = "",
        datetime_format    = "%Y-%m-%d %H:%M:%S"
    )

(datetime.datetime(2013, 1, 2, 3, 20),
 datetime.datetime(2020, 3, 31, 4, 59),
 1230779)